In [1]:
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations
import time
import os

import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics

from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

from polypharmacy.utility import *

# Train on CPU (hide GPU) due to memory constraints
#os.environ['CUDA_VISIBLE_DEVICES'] = ""

# Train on GPU
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

np.random.seed(0)

###########################################################
#
# Functions
#
###########################################################


def get_accuracy_scores(edges_pos, edges_neg, edge_type):
    feed_dict.update({placeholders['dropout']: 0})
    feed_dict.update({placeholders['batch_edge_type_idx']: minibatch.edge_type2idx[edge_type]})
    feed_dict.update({placeholders['batch_row_edge_type']: edge_type[0]})
    feed_dict.update({placeholders['batch_col_edge_type']: edge_type[1]})
    rec = sess.run(opt.preds, feed_dict=feed_dict)

    def sigmoid(x):
        return 1. / (1 + np.exp(-x))

    # Predict on test set of edges
    preds = []
    actual = []
    predicted = []
    edge_ind = 0
    for u, v in edges_pos[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds.append(score)

        actual.append(edge_ind)
        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_neg = []
    for u, v in edges_neg[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds_neg.append(score)

        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_all = np.hstack([preds, preds_neg])
    preds_all = np.nan_to_num(preds_all)
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    predicted = list(zip(*sorted(predicted, reverse=True, key=itemgetter(0))))[1]

    roc_sc = metrics.roc_auc_score(labels_all, preds_all)
    aupr_sc = metrics.average_precision_score(labels_all, preds_all)
    apk_sc = rank_metrics.apk(actual, predicted, k=50)

    return roc_sc, aupr_sc, apk_sc


def construct_placeholders(edge_types):
    placeholders = {
        'batch': tf.placeholder(tf.int32, name='batch'),
        'batch_edge_type_idx': tf.placeholder(tf.int32, shape=(), name='batch_edge_type_idx'),
        'batch_row_edge_type': tf.placeholder(tf.int32, shape=(), name='batch_row_edge_type'),
        'batch_col_edge_type': tf.placeholder(tf.int32, shape=(), name='batch_col_edge_type'),
        'degrees': tf.placeholder(tf.int32),
        'dropout': tf.placeholder_with_default(0., shape=()),
    }
    placeholders.update({
        'adj_mats_%d,%d,%d' % (i, j, k): tf.sparse_placeholder(tf.float32)
        for i, j in edge_types for k in range(edge_types[i,j])})
    placeholders.update({
        'feat_%d' % i: tf.sparse_placeholder(tf.float32)
        for i, _ in edge_types})
    return placeholders



In [2]:
###########################################################
#
# Load and preprocess data (This is a dummy toy example!)
#
###########################################################

####
# The following code uses artificially generated and very small networks.
# Expect less than excellent performance as these random networks do not have any interesting structure.
# The purpose of main.py is to show how to use the code!
#
# All preprocessed datasets used in the drug combination study are at: http://snap.stanford.edu/decagon:
# (1) Download datasets from http://snap.stanford.edu/decagon to your local machine.
# (2) Replace dummy toy datasets used here with the actual datasets you just downloaded.
# (3) Train & test the model.
####

val_test_size = 0.05
n_genes = 500
n_drugs = 400
n_drugdrug_rel_types = 3


gene_net, node2idx = load_ppi("data/bio-decagon-ppi.csv") # protein protein interactions
stitch2proteins = load_targets("data/bio-decagon-targets.csv") #drug protein interations
combo2stitch, combo2se, se2name = load_combo_se('data/bio-decagon-combo.csv') #Drug drug interactions



Reading: data/bio-decagon-ppi.csv
Edges: 715612
Nodes: 19081
Reading: data/bio-decagon-targets.csv
Reading: data/bio-decagon-combo.csv
Drug combinations: 63473 Side effects: 1317
Drug-drug interactions: 4649441


## Clean drug data

In [3]:
# Do some pre-processing of drug data

#count up all the unique drugs, and give them a unique id
drug2idx = {}
idx = 0
for item in combo2se: 
    drug_0 = item.split("_")[0]
    drug_1 = item.split("_")[1]
    if drug_0 not in drug2idx:
        drug2idx[drug_0] = idx
        idx+=1
    if drug_1 not in drug2idx:
        drug2idx[drug_1] = idx
        idx+=1
    
#count up all the unique side effects, give them a unique id
idx = 0
se2idx = {}
for _, se_set in combo2se.items(): 
    for se in se_set:
        if se not in se2idx:
            se2idx[se] = idx
            idx+=1
        
n_drugs = len(drug2idx)

In [4]:
# Need to create a drug - drug interaction matrix
drug_drug_adj_list = [np.zeros((len(drug2idx), len(drug2idx))) for _ in se2idx]

for drug_drug, se_set in combo2se.items():
    drug0 = drug_drug.split("_")[0]
    drug1 = drug_drug.split("_")[1]
    for se in se_set:
        se_index = se2idx[se]
        drug0_index = drug2idx[drug0]
        drug1_index = drug2idx[drug1]
        drug_drug_adj_list[se_index][drug0_index][drug1_index] = 1        
        drug_drug_adj_list[se_index][drug1_index][drug0_index] = 1
        
drug_drug_adj_list = [ sp.csr_matrix(item) for item in drug_drug_adj_list]
drug_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in drug_drug_adj_list]
        


## Work with genes (aka proteins)

In [5]:
gene_adj = nx.adjacency_matrix(gene_net)
gene_degrees = np.array(gene_adj.sum(axis=0)).squeeze()
n_genes = gene_adj.shape[0]

## Work with drug - protein interactions

In [6]:
missing = 0 
present = 0
gene_drug_matrix =  np.zeros((n_genes, n_drugs))
for (drug_id, protein_set) in stitch2proteins.items():
    drug_idx = drug2idx[drug_id]
    #print(drug_id)
    #print(drug2idx[drug_id])
    #print(protein_set)
    for item in protein_set:
        if item not in node2idx:
            missing+=1
        else:
            protein_idx = node2idx[item]
        #print(node2idx[item])
            gene_drug_matrix[protein_idx][drug_idx] = 1
            present+=1
    #break
print("Unable to find {} out of {} proteins ({})".format(missing, present, missing/present))


gene_drug_adj = sp.csr_matrix(gene_drug_matrix)
drug_gene_adj = gene_drug_adj.transpose(copy=True)

Unable to find 94 out of 18596 proteins (0.00505485050548505)


In [7]:
# data representation
adj_mats_orig = {
    (0, 0): [gene_adj, gene_adj.transpose(copy=True)],
    (0, 1): [gene_drug_adj],
    (1, 0): [drug_gene_adj],
    (1, 1): drug_drug_adj_list + [x.transpose(copy=True) for x in drug_drug_adj_list],
}
degrees = {
    0: [gene_degrees, gene_degrees],
    1: drug_degrees_list + drug_degrees_list,
}

# featureless (genes)
gene_feat = sp.identity(n_genes)
gene_nonzero_feat, gene_num_feat = gene_feat.shape
gene_feat = preprocessing.sparse_to_tuple(gene_feat.tocoo())

# features (drugs)
drug_feat = sp.identity(n_drugs)
drug_nonzero_feat, drug_num_feat = drug_feat.shape
drug_feat = preprocessing.sparse_to_tuple(drug_feat.tocoo())

# data representation
num_feat = {
    0: gene_num_feat,
    1: drug_num_feat,
}
nonzero_feat = {
    0: gene_nonzero_feat,
    1: drug_nonzero_feat,
}
feat = {
    0: gene_feat,
    1: drug_feat,
}

edge_type2dim = {k: [adj.shape for adj in adjs] for k, adjs in adj_mats_orig.items()}
edge_type2decoder = {
    (0, 0): 'bilinear',
    (0, 1): 'bilinear',
    (1, 0): 'bilinear',
    (1, 1): 'dedicom',
}

edge_types = {k: len(v) for k, v in adj_mats_orig.items()}
num_edge_types = sum(edge_types.values())
print("Edge types:", "%d" % num_edge_types)


Edge types: 2638


In [18]:
###########################################################
#
# Settings and placeholders
#
###########################################################

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_integer('neg_sample_size', 1, 'Negative sample size.')
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 50, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 64, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 32, 'Number of units in hidden layer 2.')
flags.DEFINE_float('weight_decay', 0, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_float('dropout', 0.1, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('max_margin', 0.1, 'Max margin parameter in hinge loss')
flags.DEFINE_integer('batch_size', 512, 'minibatch size.')
flags.DEFINE_boolean('bias', True, 'Bias term.')
# Important -- Do not evaluate/print validation performance every iteration as it can take
# substantial amount of time
PRINT_PROGRESS_EVERY = 150

print("Defining placeholders")
placeholders = construct_placeholders(edge_types)




DuplicateFlagError: The flag 'neg_sample_size' is defined twice. First from /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py, Second from /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py.  Description from first occurrence: Negative sample size.

In [19]:
###########################################################
#
# Create minibatch iterator, model and optimizer
#
###########################################################

print("Create minibatch iterator")
minibatch = EdgeMinibatchIterator(
    adj_mats=adj_mats_orig,
    feat=feat,
    edge_types=edge_types,
    batch_size=FLAGS.__flags["batch_size"],
    val_test_size=val_test_size
)

print("Create model")
model = DecagonModel(
    placeholders=placeholders,
    num_feat=num_feat,
    nonzero_feat=nonzero_feat,
    edge_types=edge_types,
    decoders=edge_type2decoder,
)

print("Create optimizer")
with tf.name_scope('optimizer'):
    opt = DecagonOptimizer(
        embeddings=model.embeddings,
        latent_inters=model.latent_inters,
        latent_varies=model.latent_varies,
        degrees=degrees,
        edge_types=edge_types,
        edge_type2dim=edge_type2dim,
        placeholders=placeholders,
        batch_size=FLAGS.__flags["batch_size"],
        margin=FLAGS.__flags["max_margin"]
    )

print("Initialize session")
sess = tf.Session()
sess.run(tf.global_variables_initializer())
feed_dict = {}

Create minibatch iterator
Minibatch edge type: (0, 0, 0)
Constructing test edges= 0000/71561
Constructing test edges= 1000/71561
Constructing test edges= 2000/71561
Constructing test edges= 3000/71561
Constructing test edges= 4000/71561
Constructing test edges= 5000/71561
Constructing test edges= 6000/71561
Constructing test edges= 7000/71561
Constructing test edges= 8000/71561
Constructing test edges= 9000/71561
Constructing test edges= 10000/71561
Constructing test edges= 11000/71561
Constructing test edges= 12000/71561
Constructing test edges= 13000/71561
Constructing test edges= 14000/71561
Constructing test edges= 15000/71561
Constructing test edges= 16000/71561
Constructing test edges= 17000/71561
Constructing test edges= 18000/71561
Constructing test edges= 19000/71561
Constructing test edges= 20000/71561
Constructing test edges= 21000/71561
Constructing test edges= 22000/71561
Constructing test edges= 23000/71561
Constructing test edges= 24000/71561
Constructing test edges= 250

MemoryError: 

In [ ]:

###########################################################
#
# Train model
#
###########################################################

print("Train model")
print_every = 1
for epoch in range(FLAGS.epochs):

    minibatch.shuffle()
    itr = 0
    while not minibatch.end():
        # Construct feed dictionary
        feed_dict = minibatch.next_minibatch_feed_dict(placeholders=placeholders)
        feed_dict = minibatch.update_feed_dict(
            feed_dict=feed_dict,
            dropout=FLAGS.dropout,
            placeholders=placeholders)

        t = time.time()

        # Training step: run single weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.batch_edge_type_idx], feed_dict=feed_dict)
        train_cost = outs[1]
        batch_edge_type = outs[2]

        if itr % print_every == 0:
            val_auc, val_auprc, val_apk = get_accuracy_scores(
                minibatch.val_edges, minibatch.val_edges_false,
                minibatch.idx2edge_type[minibatch.current_edge_type_idx])

            print("Epoch:", "%04d" % (epoch + 1), "Iter:", "%04d" % (itr + 1), "Edge:", "%04d" % batch_edge_type,
                  "train_loss=", "{:.5f}".format(train_cost),
                  "val_roc=", "{:.5f}".format(val_auc), "val_auprc=", "{:.5f}".format(val_auprc),
                  "val_apk=", "{:.5f}".format(val_apk), "time=", "{:.5f}".format(time.time() - t))

            train_auc, train_auprc, train_apk = get_accuracy_scores(
                minibatch.train_edges, minibatch.val_edges_false,
                minibatch.idx2edge_type[minibatch.current_edge_type_idx])

        itr += 1


print("Optimization finished!")

for et in range(num_edge_types):
    roc_score, auprc_score, apk_score = get_accuracy_scores(
        minibatch.test_edges, minibatch.test_edges_false, minibatch.idx2edge_type[et])
    print("Edge type=", "[%02d, %02d, %02d]" % minibatch.idx2edge_type[et])
    print("Edge type:", "%04d" % et, "Test AUROC score", "{:.5f}".format(roc_score))
    print("Edge type:", "%04d" % et, "Test AUPRC score", "{:.5f}".format(auprc_score))
    print("Edge type:", "%04d" % et, "Test AP@k score", "{:.5f}".format(apk_score))
    print()
